## 데이터 불러오기

In [86]:
import os
import pandas as pd
PATH = './data/서울시 상권분석서비스(오탈자 수정)/'

file_list= os.listdir(PATH)
csv_list = list()

for file in file_list:
    if file.split(".")[-1] == 'csv':
        csv_list.append(file)
data = ['rent','small_rent','big_rent','living_popul','indicator','resident_popul', 'income','apartment', 'area', 'store', 'working_popul', 'facilitie', 'estimated_sales']

for name, file in zip(data, csv_list):
    if name == 'rent':
        globals()[name] = pd.read_csv(PATH + file, names = ['기준년코드', '기준분기코드', '행정동코드', '행정동코드명', '임대시세층구분코드', '임대시세층구분명', '보증금평균','월임대료평균','환산임대료평균','임대건수'],encoding = "ansi")
    else:
        globals()[name] = pd.read_csv(PATH + file, encoding = "ansi")
store = store.drop("Unnamed: 0", axis = 1)
estimated_sales = estimated_sales.drop("Unnamed: 0", axis = 1)

## Data Preprocessing

### 기준 통일

In [87]:
years = [20223, 20224, 20231, 20232]

living_popul = living_popul[living_popul['기준_년분기_코드'].isin(years)]
resident_popul = resident_popul[resident_popul['기준_년분기_코드'].isin(years)]
store = store[store['기준_년분기_코드'].isin(years)]
working_popul = working_popul[working_popul['기준_년분기_코드'].isin(years)]
facilitie = facilitie[facilitie['기준_년분기_코드'].isin(years)]
estimated_sales = estimated_sales[estimated_sales['기준_년분기_코드'].isin(years)]

### 상주인구(resident_popul) 테이블

#### 불필요한 feature 제거
- 총 상주인구 수
- 남성 상주인구 수
- 여성 상주인구 수
- 연령대 10 ~ 60 이상상주인구 수
- 총 가구 수, 아파트 가구 수, 비아파트 가구수

In [88]:
resident_popul.drop(['총_상주인구_수','남성_상주인구_수','여성_상주인구_수','연령대_10_상주인구_수','연령대_20_상주인구_수','연령대_30_상주인구_수','연령대_40_상주인구_수','연령대_50_상주인구_수','연령대_60_이상_상주인구_수','총_가구_수','아파트_가구_수','비_아파트_가구_수'],axis=1,inplace=True)

In [89]:
resident_popul.head()

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,남성연령대_10_상주인구_수,남성연령대_20_상주인구_수,남성연령대_30_상주인구_수,남성연령대_40_상주인구_수,남성연령대_50_상주인구_수,남성연령대_60_이상_상주인구_수,여성연령대_10_상주인구_수,여성연령대_20_상주인구_수,여성연령대_30_상주인구_수,여성연령대_40_상주인구_수,여성연령대_50_상주인구_수,여성연령대_60_이상_상주인구_수
15015,20223,A,골목상권,3110001,이북5도청사,85,64,44,68,84,121,75,78,50,92,110,149
15016,20223,A,골목상권,3110002,독립문역 1번,39,29,30,45,51,115,40,28,29,33,40,150
15017,20223,A,골목상권,3110003,세검정초등학교,285,203,179,256,234,348,318,196,193,295,299,406
15018,20223,A,골목상권,3110004,대신고등학교,30,34,24,33,33,67,27,27,23,36,44,93
15019,20223,A,골목상권,3110005,세검정,205,153,119,152,173,242,151,141,131,194,201,291


#### 결측치 확인 및 제거
- 결측값 없음

In [90]:
resident_popul.isnull().sum()

기준_년분기_코드             0
상권_구분_코드              0
상권_구분_코드_명            0
상권_코드                 0
상권_코드_명               0
남성연령대_10_상주인구_수       0
남성연령대_20_상주인구_수       0
남성연령대_30_상주인구_수       0
남성연령대_40_상주인구_수       0
남성연령대_50_상주인구_수       0
남성연령대_60_이상_상주인구_수    0
여성연령대_10_상주인구_수       0
여성연령대_20_상주인구_수       0
여성연령대_30_상주인구_수       0
여성연령대_40_상주인구_수       0
여성연령대_50_상주인구_수       0
여성연령대_60_이상_상주인구_수    0
dtype: int64

#### 이상치 확인 및 제거
- 이상치가 없다고 판단됨

In [91]:
resident_popul.describe()

,기준_년분기_코드,상권_코드,남성연령대_10_상주인구_수,남성연령대_20_상주인구_수,남성연령대_30_상주인구_수,남성연령대_40_상주인구_수,남성연령대_50_상주인구_수,남성연령대_60_이상_상주인구_수,여성연령대_10_상주인구_수,여성연령대_20_상주인구_수,여성연령대_30_상주인구_수,여성연령대_40_상주인구_수,여성연령대_50_상주인구_수,여성연령대_60_이상_상주인구_수
count,6512.000000,6.512000e+03,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000,6512.000000
mean,20227.495393,3.115075e+06,118.175369,147.964373,210.037623,193.310350,188.114251,309.056204,111.979423,152.207002,206.473434,179.830006,184.967905,358.887285
std,4.031893,1.028490e+04,134.548401,165.438433,231.487150,204.474879,187.631109,305.687608,128.133455,179.216010,231.099368,193.030241,190.433454,359.222605
min,20223.000000,3.001491e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20223.000000,3.110402e+06,26.000000,34.000000,52.000000,50.000000,51.000000,84.000000,25.000000,35.000000,49.000000,44.000000,47.000000,95.000000
50%,20224.000000,3.110809e+06,75.500000,95.000000,138.000000,127.000000,130.000000,213.000000,71.000000,96.000000,135.000000,118.000000,124.000000,247.000000
75%,20231.000000,3.120131e+06,164.000000,208.000000,290.000000,270.000000,266.000000,434.000000,155.000000,207.000000,285.250000,249.000000,263.250000,507.000000
max,20232.000000,3.130327e+06,1586.000000,2161.000000,2148.000000,2235.000000,1577.000000,2260.000000,1492.000000,2812.000000,2070.000000,2045.000000,1584.000000,2921.000000


#### 중복행 확인 및 제거
- 중복행 없음

In [92]:
resident_popul[resident_popul.duplicated()]

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,남성연령대_10_상주인구_수,남성연령대_20_상주인구_수,남성연령대_30_상주인구_수,남성연령대_40_상주인구_수,남성연령대_50_상주인구_수,남성연령대_60_이상_상주인구_수,여성연령대_10_상주인구_수,여성연령대_20_상주인구_수,여성연령대_30_상주인구_수,여성연령대_40_상주인구_수,여성연령대_50_상주인구_수,여성연령대_60_이상_상주인구_수


#### 분기데이터 1년치 데이터로 변환

In [93]:
resident_popul['기준_년분기_코드'].value_counts()

기준_년분기_코드
20223    1634
20231    1628
20224    1625
20232    1625
Name: count, dtype: int64

In [94]:
resident_popul.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6512 entries, 15015 to 24485
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   기준_년분기_코드           6512 non-null   int64 
 1   상권_구분_코드            6512 non-null   object
 2   상권_구분_코드_명          6512 non-null   object
 3   상권_코드               6512 non-null   int64 
 4   상권_코드_명             6512 non-null   object
 5   남성연령대_10_상주인구_수     6512 non-null   int64 
 6   남성연령대_20_상주인구_수     6512 non-null   int64 
 7   남성연령대_30_상주인구_수     6512 non-null   int64 
 8   남성연령대_40_상주인구_수     6512 non-null   int64 
 9   남성연령대_50_상주인구_수     6512 non-null   int64 
 10  남성연령대_60_이상_상주인구_수  6512 non-null   int64 
 11  여성연령대_10_상주인구_수     6512 non-null   int64 
 12  여성연령대_20_상주인구_수     6512 non-null   int64 
 13  여성연령대_30_상주인구_수     6512 non-null   int64 
 14  여성연령대_40_상주인구_수     6512 non-null   int64 
 15  여성연령대_50_상주인구_수     6512 non-null   int64 
 16  여성연령대_60_이상_상주인구_수  6512

In [95]:
resident_popul_year = resident_popul[resident_popul['기준_년분기_코드'] == 20232]

In [97]:
resident_popul_year.drop(['상권_구분_코드','기준_년분기_코드','상권_코드'],axis=1,inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_17604\122505451.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resident_popul_year.drop(['상권_구분_코드','기준_년분기_코드','상권_코드'],axis=1,inplace=True)


In [98]:
resident_popul_year.head()

,상권_구분_코드_명,상권_코드_명,남성연령대_10_상주인구_수,남성연령대_20_상주인구_수,남성연령대_30_상주인구_수,남성연령대_40_상주인구_수,남성연령대_50_상주인구_수,남성연령대_60_이상_상주인구_수,여성연령대_10_상주인구_수,여성연령대_20_상주인구_수,여성연령대_30_상주인구_수,여성연령대_40_상주인구_수,여성연령대_50_상주인구_수,여성연령대_60_이상_상주인구_수
19887,골목상권,이북5도청사,66,59,70,53,73,154,50,71,52,66,98,199
19888,골목상권,독립문역 1번,30,31,23,44,58,140,22,29,30,35,53,168
19889,골목상권,세검정초등학교,165,206,210,180,237,419,166,207,171,201,296,468
19890,골목상권,대신고등학교,28,32,28,39,49,83,23,23,29,39,45,111
19891,골목상권,세검정,133,171,138,124,181,299,110,137,129,161,229,341


### 직장인구(working_popul) 테이블

#### 불필요한 feature 제거
- 총 상주인구 수
- 남성 상주인구 수
- 여성 상주인구 수
- 연령대 10 ~ 60 이상상주인구 수

In [104]:
working_popul.drop(['총_직장_인구_수','남성_직장_인구_수','여성_직장_인구_수','연령대_10_직장_인구_수','연령대_20_직장_인구_수','연령대_30_직장_인구_수','연령대_40_직장_인구_수','연령대_50_직장_인구_수','연령대_60_이상_직장_인구_수'],axis=1,inplace=True)

#### 결측치 확인 및 제거
- 결측치 없음

In [105]:
working_popul.isnull().sum()

기준_년분기_코드              0
상권_구분_코드               0
상권_구분_코드_명             0
상권_코드                  0
상권_코드_명                0
남성연령대_10_직장_인구_수       0
남성연령대_20_직장_인구_수       0
남성연령대_30_직장_인구_수       0
남성연령대_40_직장_인구_수       0
남성연령대_50_직장_인구_수       0
남성연령대_60_이상_직장_인구_수    0
여성연령대_10_직장_인구_수       0
여성연령대_20_직장_인구_수       0
여성연령대_30_직장_인구_수       0
여성연령대_40_직장_인구_수       0
여성연령대_50_직장_인구_수       0
여성연령대_60_이상_직장_인구_수    0
dtype: int64

#### 이상치 확인 및 제거

In [106]:
working_popul.describe()

,기준_년분기_코드,상권_코드,남성연령대_10_직장_인구_수,남성연령대_20_직장_인구_수,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수
count,6492.000000,6.492000e+03,6492.000000,6492.000000,6492.000000,6492.000000,6492.000000,6492.000000,6492.00000,6492.000000,6492.000000,6492.000000,6492.000000,6492.000000
mean,20227.500000,3.115039e+06,3.224892,146.669131,290.178681,278.334566,203.295132,119.865681,4.24276,188.491682,204.022181,155.911892,119.256315,57.931608
std,4.031439,1.025863e+04,18.610505,582.303084,1158.498075,1061.613471,684.319228,302.362544,24.23879,814.910146,848.973083,559.551174,426.724668,174.106291
min,20223.000000,3.001491e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20223.750000,3.110402e+06,0.000000,4.000000,9.000000,13.000000,13.000000,10.000000,0.00000,5.000000,8.000000,11.000000,9.000000,4.000000
50%,20227.500000,3.110810e+06,0.000000,14.000000,31.000000,42.000000,41.000000,32.000000,0.00000,19.000000,25.000000,31.000000,29.000000,13.000000
75%,20231.250000,3.120128e+06,1.000000,54.000000,117.000000,129.000000,118.000000,97.000000,1.00000,63.000000,79.000000,79.000000,70.000000,39.000000
max,20232.000000,3.130327e+06,589.000000,8860.000000,17233.000000,15133.000000,10824.000000,5143.000000,674.00000,18274.000000,17927.000000,8251.000000,8019.000000,2130.000000


#### 중복행 확인 및 제거
- 중복행 없음

In [107]:
working_popul[working_popul.duplicated()]

,기준_년분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,남성연령대_10_직장_인구_수,남성연령대_20_직장_인구_수,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수


#### 분기데이터 1년치 데이터로 변환

In [108]:
working_popul['기준_년분기_코드'].value_counts()

기준_년분기_코드
20223    1623
20224    1623
20231    1623
20232    1623
Name: count, dtype: int64

In [109]:
working_popul_year = working_popul[working_popul['기준_년분기_코드'] == 20232]

In [110]:
working_popul_year.drop(['상권_구분_코드','기준_년분기_코드','상권_코드'],axis=1,inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_17604\1421352378.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_popul_year.drop(['상권_구분_코드','기준_년분기_코드','상권_코드'],axis=1,inplace=True)


In [111]:
working_popul_year.head()

,상권_구분_코드_명,상권_코드_명,남성연령대_10_직장_인구_수,남성연령대_20_직장_인구_수,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수
11546,발달상권,북촌(안국역),5,644,2891,2647,1702,393,3,504,745,535,262,112
11547,골목상권,중앙고등학교,0,27,55,100,80,146,0,65,111,109,80,56
11548,골목상권,부암동주민센터,0,2,5,2,3,4,0,0,2,5,2,2
11549,골목상권,성곡미술관,0,70,152,149,69,49,0,190,177,111,42,33
11550,발달상권,서촌(경복궁역),0,190,387,258,131,105,37,297,287,163,90,44


### 임대료 데이터

In [48]:
small_rent

,상권별(1),상권별(2),상권별(3),2023.2/4,2023.3/4
0,전국,소계,소계,19.4,19.4
1,서울,소계,소계,49.2,49.5
2,서울,소계(도심지역),소계,66.4,66.8
3,서울,소계(도심지역),광화문,82.5,83.0
4,서울,소계(도심지역),남대문,63.9,64.0
...,...,...,...,...,...
249,제주,소계,소계,14.3,14.5
250,제주,광양사거리,소계,12.3,12.3
251,제주,노형오거리,소계,20.1,20.2
252,제주,서귀포도심,소계,17.0,17.4


In [49]:
area.head()

,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,자치구_코드,자치구_코드_명,행정동_코드,행정동_코드_명,영역_면적
0,A,골목상권,3110008,배화여자대학교(박노수미술관),197093,453418,11110,종로구,11110515,청운효자동,149264
1,A,골목상권,3110009,자하문터널,196991,455057,11110,종로구,11110550,부암동,178306
2,A,골목상권,3110010,평창동서측,197064,456643,11110,종로구,11110560,평창동,369415
3,A,골목상권,3110017,정독도서관,198581,453781,11110,종로구,11110600,가회동,83855
4,A,골목상권,3110018,중앙고등학교,198883,453690,11110,종로구,11110600,가회동,166872


In [50]:
rent.head()

,기준년코드,기준분기코드,행정동코드,행정동코드명,임대시세층구분코드,임대시세층구분명,보증금평균,월임대료평균,환산임대료평균,임대건수
0,2022,4,11650660,내곡동,1,1층,3286492,124345,157210,13
1,2022,4,11650660,내곡동,2,1층외,17778757,110169,287957,8
2,2022,4,11650660,내곡동,0,전체층,8807355,118945,207018,21
3,2022,4,11650620,방배3동,2,1층외,3590641,92720,128626,12
4,2022,4,11650620,방배3동,0,전체층,2245880,93565,116024,26
